ANACHRONOX BSP UPSCALE PREPARER

About the notebook:

As discussed in this moddb article (https://www.moddb.com/mods/anachronox-graphical-enhancements/news/upscaling-anachronox-with-esrgan)
there are hurdles when it comes to using high res textures for Anachronox' level architecture. In short: The game is able to load higher 
res textures, but doesn't squish them down so they occupy the same space as their low res counterparts. 

Three theoretical solutions where proposed to this problem. The one implemented in this notebook is a variation of the first:

The idea is to decompile the original .bsp files back to the .map format used when crafting levels. Then use a level editor to fix 
all textures by hand and then recomplile to create versions that work with scaled textures instead of the original ones.

Instead of decompiling the bsps, this notebook changes the texture projection data directly in the bsp file. It's the same procedure
for every textured face of the level so it can be easily automated.

ATTENTION:
A bsp modified by this notebook will have messed up lightmaps. To fix this fetch "arad3.exe" from the official Anachronox modding tools
(https://www.moddb.com/games/anachronox/downloads/anachronox-modding-tools) and make it rebuild the lightmaps.

Use it with the follwing commandline (where yourfile.bsp is to be substituted with your bsp, duh! :) ) to fix the issue:

arad3 -threads 4 -chop 256 -bounce 0 yourfile.bsp  


----


USER INPUT:

In [ ]:
# change to the filename of your bsp:
mapname = 'whacks.bsp' 

# change to your input and output path (remember to use \\ instead of \ because of pyhtons sting conventions):
input_path = 'E:\\Spiele\\Steam\\SteamApps\\common\\Anachronox\\anoxdata\\maps\\' 
output_path = 'E:\\Spiele\\Steam\\SteamApps\\common\\Anachronox\\anoxdata\\maps\\scaled\\' 

# change to the desired texture-scaling factor you want to use. (2 for 2x textures 4 for 4x etc):
scaler = 4 

----

PROCESS:

In [66]:
#Open the BSP:

file = open(input_path + mapname,'rb')


In [68]:
#read the whole bsp into memory
bsp = bytearray(file.read())



In [69]:
#fetch bsp header data
#Does a bit more than needed. Theoretically going directly
#for the "texinfo" stuff would be enough. 
file.seek(0)

magic = file.read(4)
version = int.from_bytes(file.read(4), byteorder='little')

entities_offset = int.from_bytes(file.read(4), byteorder='little')
entities_length = int.from_bytes(file.read(4), byteorder='little')

planes_offset = int.from_bytes(file.read(4), byteorder='little')
planes_length = int.from_bytes(file.read(4), byteorder='little')

vertices_offset = int.from_bytes(file.read(4), byteorder='little')
vertices_length = int.from_bytes(file.read(4), byteorder='little')

visibility_offset = int.from_bytes(file.read(4), byteorder='little')
visibility_length = int.from_bytes(file.read(4), byteorder='little')

nodes_offset = int.from_bytes(file.read(4), byteorder='little')
nodes_length = int.from_bytes(file.read(4), byteorder='little')

texinfo_offset = int.from_bytes(file.read(4), byteorder='little')
texinfo_length = int.from_bytes(file.read(4), byteorder='little')

faces_offset = int.from_bytes(file.read(4), byteorder='little')
faces_length = int.from_bytes(file.read(4), byteorder='little')

lightmaps_offset = int.from_bytes(file.read(4), byteorder='little')
lightmaps_length = int.from_bytes(file.read(4), byteorder='little')

leaves_offset = int.from_bytes(file.read(4), byteorder='little')
leaves_length = int.from_bytes(file.read(4), byteorder='little')

leaffacetable_offset = int.from_bytes(file.read(4), byteorder='little')
leaffacetable_length = int.from_bytes(file.read(4), byteorder='little')

leafbrushtable_offset = int.from_bytes(file.read(4), byteorder='little')
leafbrushtable_length = int.from_bytes(file.read(4), byteorder='little')

edges_offset = int.from_bytes(file.read(4), byteorder='little')
edges_length = int.from_bytes(file.read(4), byteorder='little')

faceedgetable_offset = int.from_bytes(file.read(4), byteorder='little')
faceedgetable_length = int.from_bytes(file.read(4), byteorder='little')

models_offset = int.from_bytes(file.read(4), byteorder='little')
models_length = int.from_bytes(file.read(4), byteorder='little')

brushes_offset = int.from_bytes(file.read(4), byteorder='little')
brushes_length = int.from_bytes(file.read(4), byteorder='little')

brushsides_offset = int.from_bytes(file.read(4), byteorder='little')
brushsides_length = int.from_bytes(file.read(4), byteorder='little')

pop_offset = int.from_bytes(file.read(4), byteorder='little')
pop_length = int.from_bytes(file.read(4), byteorder='little')

areas_offset = int.from_bytes(file.read(4), byteorder='little')
areas_length = int.from_bytes(file.read(4), byteorder='little')

araportals_offset = int.from_bytes(file.read(4), byteorder='little')
araportals_length = int.from_bytes(file.read(4), byteorder='little')
file.close()

In [70]:
#This is the function used later to 
#actually scale the different texture 
#mapping variables.
#
#Assumes that a bsp is open at 'bsp'
#
#Grabs four bytes from 'offset',
#inrerprets them as float,
#multiplys their value by 'scaler'
#replaces the original value inside 'bsp'

import struct #needed for conversion of raw bytes to float and vice versa

def scale_float(offset, scaler):

    #take four bytes:
    a = bsp[offset + 0]
    b = bsp[offset + 1]
    c = bsp[offset + 2]
    d = bsp[offset + 3]

    #put them into bytes:
    temp = bytes([a,b,c,d])

    #convert bytes to float:
    u_axis = struct.unpack('f',temp)[0]

    #multiply float:
    u_axis = u_axis * scaler

    #convert float back to bytes:
    temp = bytes(struct.pack("f",u_axis))

    bsp[offset + 0] = temp[0]
    bsp[offset + 1] = temp[1]
    bsp[offset + 2] = temp[2]
    bsp[offset + 3] = temp[3]


In [75]:
#Finally: The loop

#The texture_info lump of a bsp file is composed of 
#76 bytes long chunks. One for each textured face
#in the level. Each chunk contains the uv-data for 
#the face as well as the name of the texture. We are 
#only interested in the uv data. It is composed of
#8 float values:
#
# The first three are interpreted as vector for the 
# u-axis, the forth contains the u-offset and the 
# next four do the same for the v componets.
# Prepearing the level for upscaled texutres is as 
# easy as upscaling this floats by the desired value.
#
#More detailed info about the BSP format can be found here:
#
# https://www.flipcode.com/archives/Quake_2_BSP_File_Format.shtml
#
#(And yes: It probably also works om Quake 2 BSPs. Both games use
#same BST version.)

#Detemine the number of texinfo chunks the level uses:
num_of_texinfos = int(texinfo_length / 76) 

#And get loopie:
for i in range(0, num_of_texinfos):   
    
    offset = texinfo_offset + (i * 76)
    
    scale_float(offset + 0 , scaler)  #scale u_axis(x)
    scale_float(offset + 4 , scaler)  #scale u_axis(y)
    scale_float(offset + 8 , scaler)  #scale u_axis(z)
    
    scale_float(offset + 12, scaler)  #scale u_offset
    
    scale_float(offset + 16, scaler)  #scale v_axis(x)
    scale_float(offset + 20, scaler)  #scale v_axis(y)
    scale_float(offset + 24, scaler)  #scale v_axis(z)
    
    scale_float(offset + 28, scaler)  #scale v_offset
    
    
    

In [72]:
#Save the new BSP:
outputfile = open(output_path + mapname, 'wb')
outputfile.write(bsp)
outputfile.close()